In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import h5py
import os
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import InputLayer, Input, Reshape, MaxPool2D, Conv2D, Dense, Flatten
from keras.models import Sequential

### read mnist dataset

In [2]:
dataset_path = os.path.join("..", "assignment_4")
dataset_path = os.path.join(dataset_path, "dataset")
dataset_path = os.path.join(dataset_path, "mnist.npz")

dataset = np.load(dataset_path)
x_train, y_train = dataset["x_train"], dataset["y_train"]
x_test, y_test = dataset["x_test"], dataset["y_test"]

In [3]:
print("x_train.shape", x_train.shape)
print("y_train.shape", y_train.shape)
print("x_test.shape", x_test.shape)
print("y_test.shape", y_test.shape)

x_train.shape (60000, 28, 28)
y_train.shape (60000,)
x_test.shape (10000, 28, 28)
y_test.shape (10000,)


In [5]:
img_size = x_train.shape[1]
img_size_flat = x_train.shape[1] * x_train.shape[2]
img_shape = x_train[0].shape
img_shape_full = x_train[0].reshape((x_train.shape[1], x_train.shape[2], 1)).shape
num_classes = 10
num_channels = 1

print("img_size : ", img_size)
print("img_size_flat : ", img_size_flat)
print("img_shape : ", img_shape)
print("img_shape_full : ", img_shape_full)
print("num_classes : ", num_classes)
print("num_channels : ", num_channels)

img_size :  28
img_size_flat :  784
img_shape :  (28, 28)
img_shape_full :  (28, 28, 1)
num_classes :  10
num_channels :  1


### define plot function

In [ ]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    fig, axes = plt.subplot(3,3)
    fig.subplot_adjust(hspace=0.3, wspace=0.3)
    plt.figure(figsize=(12,12))
    
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(img_shape))
            
        ax.set_xlabel("True : {}".format(cls_true[i]) if cls_pred is None else
                     "True : {}, Pred : {}".format(cls_true[i], cls_pred[i]))
        
        ax.set_xticks([])
        ax.set_yticks([])
        
    plt.show()
    

